First, clone the Yana's **manopth** and Omid's **MANO** into `./thirdparty`

manopth: https://github.com/hassony2/manopth, SHA `4f1dcad1201ff1bfca6e065a85f0e3456e1aa32b`  
MANO: https://github.com/otaheri/MANO,  SHA `5869ab059c1bf31cc724f57eaf93e041135e8960`

In [1]:
import os, sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
import numpy as np
import thirdparty.manopth.manopth.manolayer as YanaMano
import thirdparty.MANO.mano as OmidMano
import manotorch.manolayer as ThisMano


### test PCA mode

In [2]:
batch_size = 10
ncomps = 15

seed = 23
np.random.seed(seed)
torch.manual_seed(seed)

random_shape = torch.rand(batch_size, 10)
# Generate random pose parameters, including 3 values for global axis-angle rotation
random_pose = torch.rand(batch_size, 3 + ncomps)

In [3]:
yana_mano_layer = YanaMano.ManoLayer(
    center_idx=None,
    side="right",
    mano_root="../assets/mano/models/",
    use_pca=True,
    flat_hand_mean=True,
    ncomps=ncomps
)

# hand's vertices and joints. The unit set by Yana is millimeters.
V1, J1 = yana_mano_layer(random_pose, random_shape)
# change to meter, 
V1 = V1 / 1000.0  
J1 = J1 / 1000.0

this_mano_layer = ThisMano.ManoLayer(
    rot_mot="axisang",
    center_idx=None,
    side="right",
    mano_assets_root="../assets/mano",
    use_pca=True,
    flat_hand_mean=True,
    ncomps=ncomps
)

out = this_mano_layer(random_pose, random_shape)
V2, J2 = out.verts, out.joints

assert torch.allclose(V1, V2, atol=1e-6) # less than 0.001 mm is acceptable
assert torch.allclose(J1, J2, atol=1e-6)


/home/lixin/Doc/manotorch/thirdparty/manopth/manopth/manolayer.py:72: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /opt/conda/conda-bld/pytorch_1678402312629/work/torch/csrc/utils/tensor_numpy.cpp:206.)
  torch.Tensor(smpl_data['betas'].r).unsqueeze(0))


In [4]:
omid_mano_model = OmidMano.load(model_path="../assets/mano/models",
                     is_rhand= True,
                     num_pca_comps=ncomps,
                     batch_size=batch_size,
                     flat_hand_mean=True)

output = omid_mano_model(
    global_orient=random_pose[:,:3],
    hand_pose=random_pose[:,3:],
    transl = None,
    betas=random_shape, 
    return_verts=True,
    return_tips = True
)

V3, J3 = output.vertices, output.joints
assert torch.allclose(V2, V3, atol=1e-6) # less than 0.001 mm is acceptable

In Omid's MANO model, the definitions of Joint order and Joint tips index differ from those of Yana's and ours.  
However, converting between them is quite simple.



In [5]:
OURS_TIP_IDS = {
    'thumb':		745,
    'index':		317,
    'middle':		444,
    'ring':		    556,
    'pinky':		673,
}
REORDER_IDX = [0, 13, 14, 15, 16, 1, 2, 3, 17, 4, 5, 6, 18, 10, 11, 12, 19, 7, 8, 9, 20]

def add_tips(vertices, joints, joint_ids = None):
    if joint_ids is None:
        joint_ids = torch.tensor(list(OURS_TIP_IDS.values()),
                                dtype=torch.long)
    extra_joints = torch.index_select(vertices, 1, joint_ids)
    joints = torch.cat([joints, extra_joints], dim=1)

    return joints

J3_ = J3[:, :16] # remove the tips in Omid's definition
tips = V3[:, [745, 317, 444, 556, 673]] # get the tips from Yana's definition
J3_ = torch.cat([J3_, tips], 1)
new_J3 = J3_[:, REORDER_IDX] # reorder the joints to match Yana's definition

assert torch.allclose(J2, new_J3, atol=1e-6)  # less than 0.001 mm is acceptable